<a href="https://colab.research.google.com/github/noneotherthanmodi/zenML/blob/main/zenML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install zenml

In [5]:
import zenml

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.


In [6]:
from zenml import pipeline

In [ ]:
%pip install "zenml[server]"

In [ ]:
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2


In [9]:
import IPython

In [10]:
NGROK_TOKEN = '2bgFaBvfC0ozZ1EFWKe0gLSIZzi_rv3bxPpgubEKvxQcXiiT'

In [ ]:
from zenml.environment import Environment
if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

In [ ]:
!rm rf .zen
!zenml init

In [14]:
import numpy as np
import sklearn

In [15]:
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [16]:
digits = load_digits()


In [17]:
def train_test() -> None:
  digits
  data = digits.images.reshape((len(digits.images),-1))
  X_train,X_test,y_train,y_test = train_test_split(data,digits.target,test_size = 0.2,shuffle=False)
  model = SVC(gamma = 0.001)
  model.fit(X_train,y_train)
  test_acc = model.score(X_test,y_test)
  print(f"Accuracy of the model: {test_acc}")

train_test()


Accuracy of the model: 0.9583333333333334


In [18]:
from zenml import step
from typing_extensions import Annotated
from typing import Tuple
import pandas as pd


@step
def importer() -> Tuple[
    Annotated[np.ndarray,'X_train'],
    Annotated[np.ndarray,'X_test'],
    Annotated[np.ndarray,'y_train'],
    Annotated[np.ndarray,'y_test'],
]:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images),-1))
  X_train,X_test,y_train,y_test = train_test_split(data,digits.target,test_size = 0.2,shuffle=False)

  return X_train,X_test,y_train,y_test


@step
def SVC_trainer(X_train: np.ndarray,y_train: np.ndarray) -> ClassifierMixin:
  model = SVC(gamma = 0.001)
  model.fit(X_train,y_train)
  return model


@step
def evaluator(X_test: np.ndarray,y_test:np.ndarray,model:ClassifierMixin) -> float :
  test_acc = model.score(X_test,y_test)
  print(f"Accuracy of the model: {test_acc}")
  return test_acc




In [19]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  "Link all the functions to create a pipeline: "
  X_train,X_test,y_train,y_test = importer()
  model = SVC_trainer(X_train=X_train,y_train=y_train)
  evaluator(X_test=X_test,y_test=y_test,model=model)


In [20]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Step importer has started.
Step importer has finished in 2.434s.
Step SVC_trainer has started.
Step SVC_trainer has finished in 0.448s.
Step evaluator has started.
Accuracy of the model: 0.9583333333333334
Step evaluator has finished in 0.408s.
Pipeline run has finished in 3.614s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
